In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
df = pd.read_csv("sent_skytrax.csv", index_col = 0)
df = df.iloc[:,1:]

In [3]:
df.head()

,author_review_count,rating,title,date,location,body,Seat Type,Aircraft Type,Seat Layout,Date Flown,...,Sitting Comfort,Seat/bed Width,Seat/bed Length,Seat Privacy,cleaned_body,scores,pos_score,neu_score,neg_score,compound_score
0,0.0,0.3,"""seat was not comfortable""",2019-07-22,Malaysia,With their newer A320 aircraft leg room is awf...,Economy Class,A320,3x3,July 2019,...,NaN,NaN,NaN,NaN,newer aircraft leg room awful seat comfortable...,"{'neg': 0.139, 'neu': 0.604, 'pos': 0.256, 'co...",0.256,0.604,0.139,0.3626
1,0.0,0.6,"""Check in was easy""",2019-04-06,Australia,Check in was easy and boarding I was in zone o...,Economy Class,A320,3x3,April 2019,...,NaN,NaN,NaN,NaN,check easy boarding zone think paid check kg l...,"{'neg': 0.068, 'neu': 0.705, 'pos': 0.227, 'co...",0.227,0.705,0.068,0.7430
2,5.0,0.7,"""A solid experience""",2019-04-01,Australia,"A solid experience from start to finish, espec...",Economy Class,A320,3x3,March 2019,...,NaN,NaN,NaN,NaN,solid experience start finish especially given...,"{'neg': 0.051, 'neu': 0.777, 'pos': 0.172, 'co...",0.172,0.777,0.051,0.8979
3,0.0,0.1,"""my 4yr old and I allocated different seats""",2018-01-14,Australia,I discovered upon boarding that my four-year-o...,Economy Class,A330-300,3x3x3,January 2018,...,NaN,NaN,NaN,NaN,discovered upon boarding four year old allocat...,"{'neg': 0.161, 'neu': 0.726, 'pos': 0.113, 'co...",0.113,0.726,0.161,-0.5267
4,1.0,0.7,"""space between was relatively good""",2017-08-24,Indonesia,"I got 14A seat, near emergency exit door. So t...",Economy Class,A320,3x3,August 2017,...,NaN,NaN,NaN,NaN,got seat near emergency exit different standar...,"{'neg': 0.157, 'neu': 0.66, 'pos': 0.183, 'com...",0.183,0.660,0.157,0.2960


In [4]:
reviews = [r for r in df.cleaned_body.tolist()]

In [8]:
#tokenize the phrases
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

reviews_words = list(sent_to_words(reviews))

print(reviews_words[:1])

[['newer', 'aircraft', 'leg', 'room', 'awful', 'seat', 'comfortable', 'seat', 'maximization', 'mean', 'gave', 'away', 'even', 'little', 'comfort', 'passenger']]


In [9]:
#seat can be considered as a stopword
reviews_words = [[i for i in sentence if i != "seat"] for sentence in reviews_words]

In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(reviews_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[reviews_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [13]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [21]:
#NOTE: stopwords were already removed in another phase of the data analysis, so now it is not necessary
# Form Bigrams
reviews_bigr = make_bigrams(reviews_words)

## NOTE: we choose bigrams but trigrams could be tested too

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
reviews_lem = lemmatization(reviews_bigr, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(reviews_lem[:1])

[['new', 'aircraft', 'leg', 'room', 'awful', 'comfortable', 'maximization', 'mean', 'give', 'away', 'even', 'little', 'comfort', 'passenger']]


In [22]:
## Create the Dictionary and Corpus needed for Topic Modeling

# Create Dictionary
id2word = corpora.Dictionary(reviews_lem)

# Create Corpus
texts = reviews_lem

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]


Corpus maps each phrase in (word_id, word_frequency)
If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

In [23]:
id2word[0]

'aircraft'

In [28]:
# Human readable format of corpus (term-frequency)  for the first review!
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aircraft', 1),
  ('away', 1),
  ('awful', 1),
  ('comfort', 1),
  ('comfortable', 1),
  ('even', 1),
  ('give', 1),
  ('leg', 1),
  ('little', 1),
  ('maximization', 1),
  ('mean', 1),
  ('new', 1),
  ('passenger', 1),
  ('room', 1)]]

# Build the topic modeling - LDA

In [29]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [30]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.098*"still" + 0.069*"personal" + 0.054*"plenty" + 0.038*"huge" + '
  '0.031*"comfy" + 0.029*"monitor" + 0.029*"leather" + 0.027*"line" + '
  '0.026*"real" + 0.026*"main"'),
 (1,
  '0.071*"ticket" + 0.067*"family" + 0.065*"happy" + 0.062*"squeeze" + '
  '0.054*"actually" + 0.047*"believe" + 0.034*"kid" + 0.033*"plan" + '
  '0.030*"slim" + 0.029*"emergency"'),
 (2,
  '0.082*"flight" + 0.052*"economy" + 0.043*"good" + 0.041*"fly" + '
  '0.034*"airline" + 0.033*"comfortable" + 0.024*"well" + 0.023*"hour" + '
  '0.022*"legroom" + 0.022*"bad"'),
 (3,
  '0.046*"view" + 0.046*"thin" + 0.045*"become" + 0.044*"hr" + '
  '0.040*"additional" + 0.038*"unfortunately" + 0.038*"regular" + '
  '0.032*"light" + 0.031*"amount" + 0.031*"snack"'),
 (4,
  '0.414*"class" + 0.221*"business" + 0.046*"access" + 0.040*"turn" + '
  '0.025*"similar" + 0.023*"remote" + 0.020*"touchscreen" + 0.019*"direct" + '
  '0.006*"responsive" + 0.003*"suite"'),
 (5,
  '0.165*"extremely" + 0.094*"cm" + 0.053*"stuff" +

## Compute Model Perplexity and Coherence Score
Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. 
Coherence score is usually more helpful

In [32]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews_lem, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -11.086461784090512

Coherence Score:  0.39898802535563826


# Find BEST NUMBER

In [40]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print("-----------------------------------------\n Building a model with: {} topics".format(num_topics))
        model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
        
        print("Num Topics =", num_topics, " has Coherence Value of", round(coherencemodel.get_coherence(), 4))

    return model_list, coherence_values

In [41]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=reviews_lem, start=2, limit=10, step=2)

-----------------------------------------
 Building a model with: 2 topics
Num Topics = 2  has Coherence Value of 0.4061
-----------------------------------------
 Building a model with: 4 topics
Num Topics = 4  has Coherence Value of 0.4494
-----------------------------------------
 Building a model with: 6 topics
Num Topics = 6  has Coherence Value of 0.4472
-----------------------------------------
 Building a model with: 8 topics
Num Topics = 8  has Coherence Value of 0.4709


In [44]:
# the score keeps increasing, but 8 seems like a fair amount of topics to deal with

In [43]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

[(0,
  '0.126*"economy" + 0.087*"premium" + 0.032*"extra" + 0.031*"upgrade" + '
  '0.020*"money" + 0.020*"pay" + 0.018*"section" + 0.018*"worth" + '
  '0.017*"class" + 0.014*"call"'),
 (1,
  '0.060*"check" + 0.025*"boarding" + 0.023*"family" + 0.021*"unfortunately" + '
  '0.017*"helpful" + 0.017*"smooth" + 0.016*"assign" + 0.015*"baggage" + '
  '0.013*"ptv" + 0.012*"member"'),
 (2,
  '0.048*"good" + 0.038*"comfortable" + 0.034*"economy" + 0.033*"legroom" + '
  '0.023*"well" + 0.022*"screen" + 0.022*"recline" + 0.020*"aircraft" + '
  '0.020*"use" + 0.019*"space"'),
 (3,
  '0.039*"service" + 0.032*"class" + 0.032*"flight" + 0.029*"food" + '
  '0.024*"business" + 0.020*"cabin" + 0.018*"airline" + 0.017*"offer" + '
  '0.017*"emirate" + 0.017*"meal"'),
 (4,
  '0.058*"system" + 0.020*"condition" + 0.020*"date" + 0.020*"similar" + '
  '0.017*"soft" + 0.016*"touchscreen" + 0.016*"length" + 0.015*"reason" + '
  '0.015*"acceptable" + 0.013*"outbound"'),
 (5,
  '0.042*"laptop" + 0.028*"break" + 0

In [63]:
# Find dominant topic in each sentence
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=reviews):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: x[1], reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

# for row in optimal_model[corpus]:
#     print(row[0][1])
#     break
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=reviews)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,7.0,0.3508,"flight, fly, plane, hour, get, airline, passen...",newer aircraft leg room awful seat comfortable...
1,1,7.0,0.5420,"flight, fly, plane, hour, get, airline, passen...",check easy boarding zone think paid check kg l...
2,2,2.0,0.3065,"good, comfortable, economy, legroom, well, scr...",solid experience start finish especially given...
3,3,7.0,0.4050,"flight, fly, plane, hour, get, airline, passen...",discovered upon boarding four year old allocat...
4,4,2.0,0.3212,"good, comfortable, economy, legroom, well, scr...",got seat near emergency exit different standar...
5,5,2.0,0.3654,"good, comfortable, economy, legroom, well, scr...",seat pitch tight still survivable hour flight ...
6,6,7.0,0.5015,"flight, fly, plane, hour, get, airline, passen...",seat uncomfortable space seat poor hard time s...
7,7,6.0,0.4804,"leg, front, row, uncomfortable, room, make, re...",good choose airasia flight le hour distance se...
8,8,6.0,0.2576,"leg, front, row, uncomfortable, room, make, re...",allocated seat airasia flight ak nov kuala ban...
9,9,6.0,0.3411,"leg, front, row, uncomfortable, room, make, re...",seat recline notice great hate someone tilt se...


In [65]:
df_dominant_topic.index = df_dominant_topic.Document_No

In [67]:
df_dominant_topic = df_dominant_topic.drop("Document_No", axis = 1)

In [68]:
df_dominant_topic.head()

,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
Document_No,,,,
0,7.0,0.3508,"flight, fly, plane, hour, get, airline, passen...",newer aircraft leg room awful seat comfortable...
1,7.0,0.5420,"flight, fly, plane, hour, get, airline, passen...",check easy boarding zone think paid check kg l...
2,2.0,0.3065,"good, comfortable, economy, legroom, well, scr...",solid experience start finish especially given...
3,7.0,0.4050,"flight, fly, plane, hour, get, airline, passen...",discovered upon boarding four year old allocat...
4,2.0,0.3212,"good, comfortable, economy, legroom, well, scr...",got seat near emergency exit different standar...


In [69]:
len(df_dominant_topic)

2594